In [1]:
# импорт библиотек
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# загружаем датафрейм с анекдотами
df_original = pd.read_csv('posts.csv').set_index('id')
df_original

,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
3,"— Вы уверены, что хотите выключить компьютер? ..."
4,"Извините, но подписчик из меня так себе"
...,...
24651,Марк Цукерберг и Павел Дуров заходят в бар. Цу...
24652,"— Вы любите Кафку? — Да, грефневую!"
24653,На распродаже человеческих органов нaчалась да...


Некоторые матерные слова зацензурены символами * или # что усложняет обработку этих слов. Анекдотов содержащих данные символы немного, поэтому можно их и удалить. Также уберем все анекдоты содержащие английские символы.

In [3]:
df = df_original[~df_original.text.str.contains('[\*#\[\]a-zA-Z]')]
df.drop(df[df.text.str.len() < 70].index, inplace=True)
df

C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
5,"— Дед, почему наше село называется Астафьево? ..."
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ..."
...,...
24646,"Пожар в Техасе, в школе: - Кидай мне детей, я ..."
24647,Боксёры не занимаются сексом перед боем. Потом...
24648,Судья на заседании так долго призывал к порядк...


In [4]:
# для токенизации используем стеммер Портера
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) \
                   for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-ЯёЁ]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [5]:
# переводим датафрейм в словарь с текстом анекдота и с листом токенов
df_dict_duplicates = dict() 
for i, text in tqdm(df['text'].iteritems()):
    df_dict_duplicates[i] = [text, token_and_stem(text)]

# удаляем из словаря повторяющиеся анекдоты (те у кого один и тот же лист токенов)
temp = set()
df_dict = dict()
for key, value in df_dict_duplicates.items():
    if tuple(set(value[1])) not in temp:
        temp.add(tuple(set(value[1])))
        df_dict[key] = value

print("Удалено",  len(df_dict_duplicates) - len(df_dict), "дубликатов")

15583it [01:19, 196.93it/s]


Удалено 833 дубликатов


In [10]:
def dummy_fun(doc):
    return doc

stopwords = nltk.corpus.stopwords.words('russian')

# расширяем список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

# создаем лист с токенами всех анекдотов
tokenized = [value[1] for value in tqdm(df_dict.values())]

max_df = 0.6
min_df = 1e-3
max_features = 100000
ngram_range = (1, 3)
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, max_features=max_features,
                                   min_df=min_df, stop_words=stopwords,
                                   tokenizer=dummy_fun, preprocessor=dummy_fun,
                                   use_idf=True, ngram_range=ngram_range)

tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized)
tfidf_matrix

100%|██████████| 14749/14749 [00:00<00:00, 2965285.67it/s]
C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['б', 'г', 'д', 'е', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'х', 'ц', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю'] not in stop_words.
  warnings.warn(


<14749x4634 sparse matrix of type '<class 'numpy.float64'>'
	with 329170 stored elements in Compressed Sparse Row format>

In [11]:
# экспериментальном путем выведено, что оптимальное количество кластеров ~200
n_clusters = 200

agglo = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean')
answer = agglo.fit_predict(tfidf_matrix.toarray())
answer

array([  1,   1,   2, ...,   1,  17, 133], dtype=int64)

In [13]:
# добавляем в словарь значения кластеров
for i, (key, value) in enumerate(df_dict.items()):
    df_dict[key].append(answer[i])

# переводим словарь в датафрейм
df = pd.DataFrame.from_dict(df_dict, orient='index')
df.to_csv('clustered_posts.csv')
df

,0,1,2
0,"Стоят два наркомана на перроне, мимо проезжает...","[сто, два, наркома, на, перрон, мим, проезжа, ...",1
1,В советские времена объявили месяц вежливого о...,"[в, советск, врем, объяв, месяц, вежлив, обслу...",1
2,Полицейские повязали целую группу проституток ...,"[полицейск, повяза, цел, групп, проституток, с...",2
5,"— Дед, почему наше село называется Астафьево? ...","[дед, поч, наш, сел, называ, астафьев, дык, на...",79
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ...","[алл, здравств, эт, ремонт, ноутбук, да, здрав...",16
...,...,...,...
24622,- Откуда у тебя топор? - Мне Королева эльфов ...,"[откуд, у, теб, топор, мне, королев, эльф, дал...",2
24632,— Официант! Почему у меня кофе воняет хуями? —...,"[официант, поч, у, мен, коф, воня, ху, а, вы, ...",20
24637,— Давай поедем на выходные к океану. — К како...,"[дава, поед, на, выходн, к, океан, к, как, оке...",1
24639,"— Девушка, как вы прекрасны в этом вечернем ту...","[девушк, как, вы, прекрасн, в, эт, вечерн, туа...",17


In [31]:
def left_align(df: pd.DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

pd.set_option('display.max_colwidth', 0) 
df = pd.read_csv('clustered_posts.csv', index_col=0)

left_align(df[df['2'] == 75]) # анекдоты про Чапаева

,0,1,2
1489,"— Василий Иваныч, а ноги-то у тебя даже погрязнее моих! — Так я и постарше тебя буду, Петька.","['васил', 'иваныч', 'а', 'ноги-т', 'у', 'теб', 'даж', 'погрязн', 'мо', 'так', 'я', 'и', 'постарш', 'теб', 'буд', 'петьк']",75
1896,"Научил Петька Василия Иваныча водку клизмой себе заливать, чтоб изо рта запаха спиртного не было. Уединился Василий Иваныч, чтоб попробовать, а через некоторое время Петька слышит, тот орёт как резаный. - Что случилось, Василий Иваныч?! - Огурец... огурец-то как туго заходит.","['науч', 'петьк', 'васил', 'иваныч', 'водк', 'клизм', 'себ', 'залива', 'чтоб', 'из', 'рта', 'запах', 'спиртн', 'не', 'был', 'уедин', 'васил', 'иваныч', 'чтоб', 'попробова', 'а', 'через', 'некотор', 'врем', 'петьк', 'слыш', 'тот', 'орет', 'как', 'резан', 'что', 'случ', 'васил', 'иваныч', 'огурец', 'огурец-т', 'как', 'туг', 'заход']",75
2365,"- Ну и грязные же у тебя ноги, Василий Иваныч, даже грязней моих! – Дак ведь я, Петька и постарше тебя-то буду.","['ну', 'и', 'грязн', 'же', 'у', 'теб', 'ног', 'васил', 'иваныч', 'даж', 'грязн', 'мо', 'дак', 'вед', 'я', 'петьк', 'и', 'постарш', 'тебя-т', 'буд']",75
3523,"Как то у Василия Иваныча с Петькой вообще кончились деньги. даже чекушку не на что купить. Ну стали они Анку уговарвать - пойди мол к белым, ублажи одного-другого - заработаешь нам да и себе на хозяйство пару копеечек. Ну делать нечего, надо идти... приходит Анка к лагерю белых - а там патрульный - стой кто идет? Ну Анка конечно говорит так мол и так, пришла заниматься древнейшей профессией хоть за какие деньги. А патрульный в ответ - дак у меня ж денег всего 15 рублев! - Ну ладно,говорит Анка, давай за пятнадцать. Очень доволен остался солдат, хвалил Анку и порекомендовал лейтенанту. Тому тоже очень понравилось - заплатил 150 рублей да и к полковнику отправил. Полковник был в восторге - дал 1500 рублей и к генералу привел. А генерал уж так доволен был - аж 15000 пожаловал. Вот собрала Анка все эти деньги 15+150+1500+15000 плучилось аж 16665 рублей и думает - принесу их сейчас Василь Иванычу с Петькой - пропьют же они, опять на хозяйство ни гроша не останется! Спрятала 10000 а остальное в дивизию отнесла. Ну Чапаев с Петькой довольные - куча денег! Но всеж таки Чапай проверяет : - Погоди Анка, ты со сколькими то была? - Да с четырьмя Василь Иваныч. Хорошо, говорит Чапаев - четыре мужика это восемь яиц - и рисует : 0 0 0 0 0 0 0 0 Вот значит, 8 яиц. - Анка, а у них то до того стоял? - ДА да, Василь Иваныч, стоял! - Хорошо, говорит Чапаев и подрисовывает к яйцам сверху стоячие члены 6 6 6 6 - А потом значить Анка они у них то упали? - А как же Василь Иваныч, упали, донизу упали! - Хорошо, говорит Чапаев и рисует 4 упавших 9 9 9 9. Ну давай Анка посчитаем: 6666 + 9999 = 16665 Ну-ка Анка сознавайся, Куда еще 10000 тысяч девала????","['как', 'то', 'у', 'васил', 'иваныч', 'с', 'петьк', 'вообщ', 'конч', 'деньг', 'даж', 'чекушк', 'не', 'на', 'что', 'куп', 'ну', 'стал', 'он', 'анк', 'уговарва', 'пойд', 'мол', 'к', 'бел', 'ублаж', 'одного-друг', 'заработа', 'нам', 'да', 'и', 'себ', 'на', 'хозяйств', 'пар', 'копеечек', 'ну', 'дела', 'неч', 'над', 'идт', 'приход', 'анк', 'к', 'лагер', 'бел', 'а', 'там', 'патрульн', 'сто', 'кто', 'идет', 'ну', 'анк', 'конечн', 'говор', 'так', 'мол', 'и', 'так', 'пришл', 'занима', 'древн', 'професс', 'хот', 'за', 'как', 'деньг', 'а', 'патрульн', 'в', 'ответ', 'дак', 'у', 'мен', 'ж', 'денег', 'всег', 'рубл', 'ну', 'ладн', 'говор', 'анк', 'дава', 'за', 'пятнадца', 'очен', 'довол', 'оста', 'солдат', 'хвал', 'анк', 'и', 'порекомендова', 'лейтенант', 'том', 'тож', 'очен', 'понрав', 'заплат', 'рубл', 'да', 'и', 'к', 'полковник', 'отправ', 'полковник', 'был', 'в', 'восторг', 'дал', 'рубл', 'и', 'к', 'генерал', 'привел', 'а', 'генера', 'уж', 'так', 'довол', 'был', 'аж', 'пожалова', 'вот', 'собра', 'анк', 'все', 'эт', 'деньг', 'плуч', 'аж', 'рубл', 'и', 'дума', 'принес', 'их', 'сейчас', 'васил', 'иваныч', 'с', 'петьк', 'пропьют', 'же', 'он', 'опя', 'на', 'хозяйств', 'ни', 'грош', 'не', 'останет', 'спрят

In [33]:
left_align(df[df['2'] == 100]) # анекдоты про Поручика

,0,1,2
74,"Перед званым вечером у Наташи Ростовой Ржевский отчитывает гусаров: не материться, не нажираться, дам прилюдно за задницу не хватать и т. п. Видя скованность гусар за ужином, Наташа задает вопрос: «Господа гусары у меня тут есть 29 подсвечников, а я купила тридцать свечей. Куда бы вставить тридцатую?» Поручик Ржевский, вскакивая из-за стола: ""в пизду ахахаххахахааххахаххахаха""","['перед', 'зван', 'вечер', 'у', 'наташ', 'ростов', 'ржевск', 'отчитыва', 'гусар', 'не', 'матер', 'не', 'нажира', 'дам', 'прилюдн', 'за', 'задниц', 'не', 'хвата', 'и', 'т.', 'п.', 'вид', 'скован', 'гусар', 'за', 'ужин', 'наташ', 'зада', 'вопрос', 'господ', 'гусар', 'у', 'мен', 'тут', 'ест', 'подсвечник', 'а', 'я', 'куп', 'тридца', 'свеч', 'куд', 'бы', 'встав', 'тридцат', 'поручик', 'ржевск', 'вскакив', 'из-з', 'стол', 'в', 'пизд', 'ахахаххахахааххахаххахах']",100
395,"День рождения Наташи Ростовой. Наташа пригласила на праздник поручика Ржевского и всех гусаров полка. За столом Наташа пытается завести светскую беседу: - Вы знаете, я купила 17 свечек для праздничного торта, а на него влезло только 16. Ума не приложу, куда всунуть еще одну?... Гусары кричат хором: - В ПИЗДУ! Ржевский не инструктировал гусаров.","['ден', 'рожден', 'наташ', 'ростов', 'наташ', 'приглас', 'на', 'праздник', 'поручик', 'ржевск', 'и', 'всех', 'гусар', 'полк', 'за', 'стол', 'наташ', 'пыта', 'завест', 'светск', 'бесед', 'вы', 'знает', 'я', 'куп', 'свечек', 'для', 'праздничн', 'торт', 'а', 'на', 'нег', 'влезл', 'тольк', 'ум', 'не', 'прилож', 'куд', 'всунут', 'ещ', 'одн', 'гусар', 'кричат', 'хор', 'в', 'пизд', 'ржевск', 'не', 'инструктирова', 'гусар']",100
413,"Отечественная война 1812 года, гусары сидят на совете перед Бородинской битвой, курят трубки, обсуждают грядущее сражение. Входит Кутузов. – Господа гусары! До меня дошли слухи, что среди присутствующих завёлся измененник! Этот человек донёс агентам Бонапарта о грядущей расстановке войск, тем самым нарушив присягу Государю, и потому повинен смерти. Однако, нам пока так и не удалось его вычислить. Ржевский! Вы известны своим интеллектом и проницательностью, а также верностью Отечеству. Поручик, не могли бы вы показать нам, кто их присутствующих здесь – предатель? Ржевский, закинув ногу за ногу: – А могу-с.","['отечествен', 'войн', 'год', 'гусар', 'сид', 'на', 'совет', 'перед', 'бородинск', 'битв', 'кур', 'трубк', 'обсужда', 'грядущ', 'сражен', 'вход', 'кутуз', 'господ', 'гусар', 'до', 'мен', 'дошл', 'слух', 'что', 'сред', 'присутств', 'завел', 'измененник', 'этот', 'человек', 'донес', 'агент', 'бонапарт', 'о', 'грядущ', 'расстановк', 'войск', 'тем', 'сам', 'наруш', 'присяг', 'государ', 'и', 'пот', 'повин', 'смерт', 'однак', 'нам', 'пок', 'так', 'и', 'не', 'уда', 'ег', 'вычисл', 'ржевск', 'вы', 'известн', 'сво', 'интеллект', 'и', 'проницательн', 'а', 'такж', 'верност', 'отечеств', 'поручик', 'не', 'могл', 'бы', 'вы', 'показа', 'нам', 'кто', 'их', 'присутств', 'зде', 'предател', 'ржевск', 'закинув', 'ног', 'за', 'ног', 'а', 'могу-с']",100
524,"Поручик Ржевский танцует с дамой на балу. Дама томно поднимает глазки и спрашивает: – Поручик, у меня не слишком глубокое декольте? Поручик заглядывает и говорит: – А у Вас волосы на груди растут? Дама в ужасе: – Нет, что Вы! – Тогда глубокое.","['поручик', 'ржевск', 'танц', 'с', 'дам', 'на', 'бал', 'дам', 'томн', 'поднима', 'глазк', 'и', 'спрашива', 'поручик', 'у', 'мен', 'не', 'слишк', 'глубок', 'декольт', 'поручик', 'заглядыва', 'и', 'говор', 'а', 'у', 'вас', 'волос', 'на', 'груд', 'растут', 'дам', 'в', 'ужас', 'нет', 'что', 'вы', 'тогд', 'глубок']",100
1562,"Поручик Ржевский просит своего денщика: — Помоги, любезнейший! Сегодня у Наташи день рождения, я приглашен. Придумай что-нибудь веселенькое, чем бы я мог развеселить и поразить своим искрометным умом гостей! — Пожалуйста... В момент, когда задуют свечи на торте, а свет еще не включат, спросите: «Господа! Темно, как у негра где?» И всем станет весело. Поручик отправился на праздник, дождался нужного момента и, когда Нат

In [34]:
left_align(df[df['2'] == 157]) # анекдоты про Буратино

,0,1,2
1576,"Подросший Буратино спрашивает у Папы Карло: — Папа, а почему мне нравится Пьеро, а не Мальвина? — Потому что ты сделан из голубой ели!","['подросш', 'буратин', 'спрашива', 'у', 'пап', 'карл', 'пап', 'а', 'поч', 'мне', 'нрав', 'пьер', 'а', 'не', 'мальвин', 'пот', 'что', 'ты', 'сдела', 'из', 'голуб', 'ел']",157
3241,"Чтобы запастись березовым соком на всю зиму, Папа Карло всю осень давал Буратино мочегонное.","['чтоб', 'запаст', 'березов', 'сок', 'на', 'всю', 'зим', 'пап', 'карл', 'всю', 'осен', 'дава', 'буратин', 'мочегон']",157
4946,Приходит Буратино на завод. - Вы уголь принимаете? - Да. Буратино садится и начинает дрочить.,"['приход', 'буратин', 'на', 'завод', 'вы', 'угол', 'принима', 'да', 'буратин', 'сад', 'и', 'начина', 'дроч']",157
7316,"— Ты будешь мне опорой, сынок, — сказал папа Карло и переделал Буратино в костыль.","['ты', 'будеш', 'мне', 'опор', 'сынок', 'сказа', 'пап', 'карл', 'и', 'передела', 'буратин', 'в', 'костыл']",157
10631,В полумраке каморки Папа Карло наступает на кучку опилок: - Буратино! Гадина! Опять насрал!,"['в', 'полумрак', 'каморк', 'пап', 'карл', 'наступа', 'на', 'кучк', 'опилок', 'буратин', 'гадин', 'опя', 'насра']",157
10862,"Буратино в приступе ярости отпилил себе руку. И, только пересчитав годовые кольца, продавец отпустила пиво.","['буратин', 'в', 'приступ', 'ярост', 'отпил', 'себ', 'рук', 'и', 'тольк', 'пересчита', 'годов', 'кольц', 'продавец', 'отпуст', 'пив']",157
11358,"Буратино бежит по лесу и орёт: -МАЛЬВИНА ПРОПАЛА! МАЛЬВИНА ПРОПАЛА! Все ему отвечают: - Ты че дурак? Она же у тебя на руках лежит! Буратино, протягивая Мальвину в носы присутствующим: - А вы понюхайте!","['буратин', 'беж', 'по', 'лес', 'и', 'орет', '-мальвин', 'пропа', 'мальвин', 'пропа', 'все', 'ем', 'отвеча', 'ты', 'че', 'дурак', 'он', 'же', 'у', 'теб', 'на', 'рук', 'леж', 'буратин', 'протягив', 'мальвин', 'в', 'нос', 'присутств', 'а', 'вы', 'понюха']",157
13566,"Едет грузовик по проселочной дороге, в кузове сидят куча буратин. Дорога вся в ямах, машину трясёт, буратины болтаются из стороны в сторону...Вдруг один встаёт, стучит по крыше кабины-Эй, мужик, выбирай дорогу, не дрова везёшь!","['едет', 'грузовик', 'по', 'проселочн', 'дорог', 'в', 'кузов', 'сид', 'куч', 'буратин', 'дорог', 'вся', 'в', 'ям', 'машин', 'трясет', 'буратин', 'болта', 'из', 'сторон', 'в', 'сторон', 'вдруг', 'один', 'встает', 'стуч', 'по', 'крыш', 'кабины-э', 'мужик', 'выбира', 'дорог', 'не', 'дров', 'везеш']",157
14044,Артемон приходит к Буратино и говорит: - Буратино! Мальвина пропала! - Как же пропала? Вон она на травке лежит... - А ты её понюхай!!,"['артемон', 'приход', 'к', 'буратин', 'и', 'говор', 'буратин', 'мальвин', 'пропа', 'как', 'же', 'пропа', 'вон', 'он', 'на', 'травк', 'леж', 'а', 'ты', 'е', 'понюха']",157
14072,"Мальвина не раз пыталась бросить Буратино, но Артемон приносил его обратно.","['мальвин', 'не', 'раз', 'пыта', 'брос', 'буратин', 'но', 'артемон', 'принос', 'ег', 'обратн']",157


In [72]:
left_align(df[df['2'] == 108]) # анекдоты про прапора

,0,1,2
1849,"Занятие в военном училище. Прапорщик спрашивает курсанта: — Курсант Сидоров, из чего сделан затвор? — Из вороненой стали, товарищ прапорщик! — Правильно. А из чего сделан боек? — Из вороненой стали, товарищ прапорщик! — Неправильно. Плохо изучаете учебник. Там ясно написано: «Из того же материала».","['занят', 'в', 'воен', 'училищ', 'прапорщик', 'спрашива', 'курсант', 'курсант', 'сидор', 'из', 'чег', 'сдела', 'затвор', 'из', 'воронен', 'стал', 'товарищ', 'прапорщик', 'правильн', 'а', 'из', 'чег', 'сдела', 'боек', 'из', 'воронен', 'стал', 'товарищ', 'прапорщик', 'неправильн', 'плох', 'изуча', 'учебник', 'там', 'ясн', 'написа', 'из', 'тог', 'же', 'материа']",108
2686,"— Товарищ солдат! Чем это вы тут занимаетесь?! — Рисую мелом на асфальте, товарищ прапорщик, в личное время Устав не запрещает. — Ну рисуй–рисуй, Паганини.","['товарищ', 'солдат', 'чем', 'эт', 'вы', 'тут', 'занима', 'рис', 'мел', 'на', 'асфальт', 'товарищ', 'прапорщик', 'в', 'личн', 'врем', 'уста', 'не', 'запреща', 'ну', 'рисуй–рис', 'паганин']",108
2712,"- Товарищ прапорщик, что нового в роте? - Э… да так, по мелочам… зуб вырвал, пломбу поставил…","['товарищ', 'прапорщик', 'что', 'нов', 'в', 'рот', 'э…', 'да', 'так', 'по', 'мелочам…', 'зуб', 'вырва', 'пломб', 'поставил…']",108
2774,"Послали в армии на задание пятеро человек: прапорщика и четверг солдат. Закончили задание, прапорщик их считает: - Первый, второй, третий, четвертый. Так, где пятый? Все ищем пятого! Ищут они десять минут, ищут двадцать, ищут тридцать, все найти не могут. Тут один солдат говорит: -Тврщ прапорщик, давайте я посчитаю, встаньте в ряд, пжлста. Прапорщик встаёт в ряд, солдат считает: -Первый, второй, третий, четвертый и я пятый. Прапорщик резко подскакивает: -Так это мы тебя, долбоёба, полчаса искали!","['посла', 'в', 'арм', 'на', 'задан', 'пятер', 'человек', 'прапорщик', 'и', 'четверг', 'солдат', 'законч', 'задан', 'прапорщик', 'их', 'счита', 'перв', 'втор', 'трет', 'четверт', 'так', 'где', 'пят', 'все', 'ищ', 'пят', 'ищут', 'он', 'деся', 'минут', 'ищут', 'двадца', 'ищут', 'тридца', 'все', 'найт', 'не', 'могут', 'тут', 'один', 'солдат', 'говор', '-тврщ', 'прапорщик', 'дава', 'я', 'посчита', 'встаньт', 'в', 'ряд', 'пжлста', 'прапорщик', 'встает', 'в', 'ряд', 'солдат', 'счита', '-перв', 'втор', 'трет', 'четверт', 'и', 'я', 'пят', 'прапорщик', 'резк', 'подскакива', '-так', 'эт', 'мы', 'теб', 'долбоеб', 'полчас', 'иска']",108
2796,"Во время обучения в артиллерийском подразделении прапорщик рассказывает, как используется сложное прицельное устройство системы артиллерийского оружия: — Как вы все знаете, в окружности 180 градусов. Один из солдат говорит: — Но в окружности 360 градусов, товарищ прапорщик. — Ты идиот, — отвечает прапорщик, — ясно же, что я говорю о маленьком кружке!","['во', 'врем', 'обучен', 'в', 'артиллерийск', 'подразделен', 'прапорщик', 'рассказыва', 'как', 'использ', 'сложн', 'прицельн', 'устройств', 'систем', 'артиллерийск', 'оруж', 'как', 'вы', 'все', 'знает', 'в', 'окружн', 'градус', 'один', 'из', 'солдат', 'говор', 'но', 'в', 'окружн', 'градус', 'товарищ', 'прапорщик', 'ты', 'идиот', 'отвеча', 'прапорщик', 'ясн', 'же', 'что', 'я', 'говор', 'о', 'маленьк', 'кружк']",108
7703,"Сидит голубой в окопе. Кругом рвутся снаряды, мины и бомбы. Голубой подымается во весь рост и кричит: - Гады, фашисты! Такие красивые цветочки погубили! Прапорщик: - Ляг, Петухов! Ляг! Петухов: - Не лягу, товарищ прапорщик, вы мне противны!","['сид', 'голуб', 'в', 'окоп', 'круг', 'рвут', 'снаряд', 'мин', 'и', 'бомб', 'голуб', 'подыма', 'во', 'ве', 'рост', 'и', 'крич', 'гад', 'фашист', 'так', 'красив', 'цветочк', 'погуб', 'прапорщик', 'ляг', 'петух', 'ляг', 'петух', 'не', 'ляг', 'товарищ', 'прапорщик', 'вы', 'мне', 'противн']",108
7928,"Занятие в военном училище. Прапорщик спрашивает курсанта: — Курсант Сидоров, из чего сделан затвор? — Из стали, товарищ прапорщик! — Правильно. А из чего сделан боек? — Из стали, товарищ прапорщик! — Неправильно. Плохо изучаете учебник. Там ясно написано: «И